In [ ]:
# 아래 명령어를 통해 필요한 라이브러리를 미리 설치해주세요
!pip install xgboost

# 결정 트리와 앙상블 기법

**결정 트리와 앙상블 기법**은 머신러닝에서 가장 널리 쓰이는 알고리즘 중 하나입니다.  
- **결정 트리(Decision Tree)**: 구조가 직관적이고 시각화가 쉬워 해석 가능성이 높음. 그러나 단독으로는 과적합 위험이 크고 성능이 제한적일 수 있음.  
- **앙상블 기법(Ensemble Methods)**: 여러 개의 모델을 결합하여 단일 모델보다 더 좋은 성능을 발휘. 특히 배깅(Bagging)과 부스팅(Boosting)이 대표적.  

👉 이 토픽에서는 **결정 트리 기본기**부터 **랜덤 포레스트, 부스팅(XGBoost 포함)** 까지 학습하여, 복잡한 데이터셋에서도 효과적인 모델을 구축할 수 있도록 합니다.  

학습 목표

- 결정 트리의 기본 개념과 작동 방식을 이해하고 설명할 수 있다.  
- 앙상블 기법의 원리를 이해하고 실제 데이터에 적용할 수 있다.  
- 결정 트리 및 앙상블 모델을 학습·평가하고, 성능을 비교할 수 있다. 

## 0. 들어가기

머신러닝 모델 중에는 복잡하지만 해석하기 어려운 모델(예: 신경망)과  
단순하지만 이해하기 쉬운 모델(예: 선형 회귀)이 있습니다.  

👉 **결정 트리(Decision Tree)** 는 그 중간에 위치합니다.  
- 데이터의 규칙을 "나무(tree)" 구조로 표현  
- 사람이 이해하기 쉽고, 시각화 가능  
- 그러나 깊게 만들면 과적합이 쉽게 발생  

  <img src="image/Decision Tree.webp" width="500">

이미지 출처 : https://www.displayr.com/what-is-a-decision-tree/

이번 장에서는 **결정 트리의 기본 개념**부터 실습까지 다룹니다.

## 1. 결정 트리 (Decision Tree)

### 1.1 기본 개념
- 데이터를 분할하여 예측하는 알고리즘  
- 트리 구조:  
  - **루트 노드(root)**: 시작점  
  - **분기 노드(split node)**: 질문을 던지는 부분 (예: 나이 > 30?)  
  - **리프 노드(leaf node)**: 최종 예측 결과 (예: 생존 / 사망)  

  <img src="image/tree_arch.webp">  
  
  이미지 출처 : https://medium.com/@ryassminh/math-for-ml-understand-regression-decision-trees-with-simple-examples-9474fceb6802

### 1.2 작동 방식
1. 데이터를 가장 잘 나누는 기준(feature, 조건)을 선택  
2. 해당 조건으로 분할  
3. 각 분할된 그룹에 대해 다시 같은 과정을 반복  
4. 더 이상 나눌 수 없을 때 예측 결과 결정  

👉 즉, "질문을 반복하면서 최종 답을 찾는 과정"

### 1.3 분할 기준

결정 트리(Decision Tree)는 데이터를 나눌 때 **불순도(impurity)** 를 줄이는 방향으로 작동합니다.  
즉, 노드를 분할했을 때 데이터가 더 “순수(pure)”해지도록 기준을 정하는 것입니다.  


####  불순도(Impurity)란?
- **불순도 = 섞여 있는 정도**
- 한 노드에 클래스가 여러 개 뒤섞여 있으면 **불순도↑**
- 한 노드에 모두 같은 클래스만 있으면 **불순도=0 (완전 순수)**

  $$
  G(p) = 1 - \sum_{i=1}^k p_i^2
  $$
  
#### 예시: 남/여 구분

어떤 데이터셋이 있고, 트리가 사람의 성별(남자/여자)을 맞추는 문제라고 해봅시다.

####  루트 노드 (처음 상태)
- 데이터: 총 10명  
  - 남자(Male) = 5명  
  - 여자(Female) = 5명  

- 지니 불순도 계산:
  $$
  G = 1 - (p_{male}^2 + p_{female}^2) 
  = 1 - (0.5^2 + 0.5^2) 
  = 1 - (0.25+0.25) = 0.5
  $$
  → 완전히 섞여 있음 (불순도 최대)


####  분할 1: “머리 길이” 기준으로 나누기
- **긴 머리 그룹**: 여자 4명, 남자 1명  
  $$
  G = 1 - (0.8^2 + 0.2^2) 
  = 1 - (0.64+0.04) = 0.32
  $$

- **짧은 머리 그룹**: 남자 4명, 여자 1명  
  $$
  G = 1 - (0.8^2 + 0.2^2) = 0.32
  $$

👉 분할 후 평균 지니 불순도:
$$
G_{avg} = \frac{5}{10}\times 0.32 + \frac{5}{10}\times 0.32 = 0.32
$$

→ 루트에서 0.5였던 불순도가 0.32로 **줄어듦** → 좋은 분할!


####  분할 2: “키” 기준으로 나누기
- **키 ≥ 170cm 그룹**: 남자 3명, 여자 2명  
  $$
  G = 1 - (0.6^2 + 0.4^2) = 1 - (0.36+0.16) = 0.48
  $$
- **키 < 170cm 그룹**: 남자 2명, 여자 3명  
  $$
  G = 1 - (0.4^2 + 0.6^2) = 0.48
  $$

👉 분할 후 평균 지니 불순도:
$$
G_{avg} = 0.48
$$

→ 루트에서 0.5였던 불순도가 0.48로 **거의 줄지 않음** → 나쁜 분할.

#### 🔹 자동으로 분할 기준 선택
- 사람이 `170cm일까, 180cm일까?`라고 임의로 정하는 게 아니라,  
- **트리 알고리즘이 모든 feature와 가능한 분할 기준(threshold)** 을 시도해봅니다.  
  - 연속형 변수(키, 나이, 수입): 가능한 분할점 후보를 전부 계산 → 불순도 감소량이 최대인 기준 선택  
  - 범주형 변수(성별, 지역): 가능한 조합을 모두 시도 → 불순도 감소량이 최대인 조합 선택  
- 결과적으로 “불순도를 가장 크게 줄여주는 기준”이 자동으로 선택됩니다.


### ✅ 정리
- 트리는 **여러 후보 특성(feature)** 중에서  
  **불순도를 가장 크게 줄이는 기준**을 스스로 선택해 데이터를 나눕니다.
- 예제에서는 “머리 길이”가 “키”보다 훨씬 좋은 분할 기준.


### 💡 직관적 비유
- 반 친구들을 남/여로 정확히 나누고 싶다.  
- **키 기준**으로 나누면 섞임이 여전히 많음 → 불순도 높음.  
- **머리 길이 기준**으로 나누면 각 그룹이 한쪽 성별로 치우침 → 불순도 낮음.  
- 따라서 트리는 “머리 길이”를 자동으로 최적 기준으로 선택.
```

### 1) 엔트로피(Entropy)
- 데이터의 **무질서 정도**를 측정
- 공식:
  $$
  H(p) = - \sum p \log_2 p
  $$
  여기서 \(p\)는 특정 클래스에 속할 확률

- 👉 값 해석:
  - 클래스가 섞여 있을수록 엔트로피 ↑ (최대 = 1)
  - 한쪽 클래스만 있으면 엔트로피 = 0

| 예시 (두 클래스 비율) | 엔트로피 값 |
|-------------------|-----------|
| (0, 1) → 한쪽만 존재 | 0 |
| (0.5, 0.5) → 반반   | 1 (최대 무질서) |
| (0.8, 0.2)         | 0.72 |

### 2) 지니 불순도(Gini Impurity)
- 무작위로 두 샘플을 뽑았을 때 **서로 다른 클래스일 확률**
- 공식:
  $$
  G(p) = 1 - \sum_{i=1}^k p_i^2
  $$

- $(k$): 클래스의 개수  
- $(p_i$): 현재 노드에서 클래스 $(i$)에 속할 확률 (샘플 비율)

### 🔹 유도 과정 (직관)
1. 어떤 노드에 클래스 A, B가 있다고 가정
   - 클래스 A 비율 = $(p_A$)  
   - 클래스 B 비율 = $(p_B$)  
   - (조건: $(p_A + p_B = 1$))

2. 무작위로 두 샘플을 뽑았을 때 **같은 클래스일 확률**은?  
   - 둘 다 A일 확률: $(p_A \times p_A = p_A^2$)  
   - 둘 다 B일 확률: $(p_B \times p_B = p_B^2$)  
   - 따라서 "같은 클래스일 확률" = $(p_A^2 + p_B^2$)

3. "서로 다른 클래스일 확률" = $(1 - (p_A^2 + p_B^2)$)  
   → 이것이 바로 **지니 불순도** 공식!


### 🔹 예시 (2클래스)
- (0.5, 0.5):  
  $(G = 1 - (0.5^2 + 0.5^2) = 1 - (0.25+0.25) = 0.5$)  
  → 가장 섞여 있음 (불순도 최대)

- (0.8, 0.2):  
  $(G = 1 - (0.64 + 0.04) = 0.32$)  
  → 다소 순수해짐

- (1.0, 0.0):  
  $(G = 1 - (1^2 + 0^2) = 0$)  
  → 완전히 순수


### 🔹 특징
- 범위: 0 ~ 0.5 (2클래스일 때)  
- 클래스가 많아질수록 최대값은 1에 가까워짐 (더 섞일 수 있으므로)  
- **작을수록 순수** → 결정 트리는 Gini가 줄어드는 방향으로 분할을 선택

- 👉 값 해석:
  - 클래스가 섞일수록 지니 ↑ (최대 = 0.5 for 2클래스)
  - 한쪽 클래스만 있으면 지니 = 0

| 예시 (두 클래스 비율) | 지니 값 |
|-------------------|--------|
| (0, 1)            | 0 |
| (0.5, 0.5)        | 0.5 (최대) |
| (0.8, 0.2)        | 0.32 |

### 엔트로피 vs 지니 비교
- 둘 다 **데이터 분할의 불순도**를 측정
- **공통점**: 한쪽으로 치우칠수록 값 ↓, 섞일수록 값 ↑
- **차이점**:
  - 엔트로피: 로그 계산 → 이론적 의미(정보량) 강조
  - 지니: 계산 단순, 조금 더 빠름
- 실제 성능 차이는 크지 않음 (둘 다 많이 사용)


### 직관적 비유
- 반에서 학생들이 "사과팀"과 "바나나팀"으로 나뉜다고 하자.
- **한 반에 모두 사과팀만 있다** → 완전히 순수, 불순도=0
- **사과팀/바나나팀이 반반** → 가장 섞여 있음, 불순도 최대
- 결정 트리는 이런 불순도를 줄이는 쪽으로 반을 나누어감.

### 1.4 Scikit-learn으로 결정 트리 분류기 만들기

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt

# 데이터 불러오기
data = load_breast_cancer()
X, y = data.data, data.target

# 결정 트리 모델 학습
model = DecisionTreeClassifier(max_depth=3, random_state=42)
model.fit(X, y)

# 트리 시각화
plt.figure(figsize=(12, 6))
plot_tree(model, feature_names=data.feature_names, class_names=data.target_names, filled=True)
plt.show()

### 1.5 가지치기 (Pruning)
- 트리가 너무 깊어지면 훈련 데이터에 과적합됨  
- 해결 방법:  
  - `max_depth`: 트리의 최대 깊이 제한  
  - `min_samples_split`: 분할하기 위한 최소 샘플 수  
  - `min_samples_leaf`: 리프 노드에 필요한 최소 샘플 수  
  
  <img src="image/Prune.png">  

  이미지 출처 : https://developers.google.com/machine-learning/decision-forests/overfitting-and-pruning?hl=ko

### ✅ 체크포인트
- 결정 트리는 데이터를 조건으로 분할하는 알고리즘이다.  
- 분할 기준으로는 엔트로피와 지니 불순도가 사용된다.  
- 트리가 너무 깊으면 과적합되므로 가지치기로 제어해야 한다.  
- Scikit-learn을 사용하면 결정 트리 모델을 손쉽게 구현할 수 있다.  


## 2. 앙상블 기법 (Ensemble Methods)

### 2.1 앙상블 기법이란?
- 단일 모델 하나보다, 여러 개의 모델을 결합하면 더 좋은 성능을 낼 수 있습니다.  
- 이를 **앙상블(ensemble)** 기법이라고 합니다.  
- 아이디어: "여러 사람이 투표하면 더 정확한 답을 얻을 수 있다."  


### 2.2 배깅 (Bagging: Bootstrap Aggregating)
- 데이터 샘플을 여러 번 복원추출(bootstrap) 각각의 모델을 학습시킴  
- 모든 모델의 예측을 `평균(회귀)` 또는 `다수결(분류)`로 결합  
- 분산(Variance)을 줄이고, 과적합 위험을 완화  

👉 대표적인 배깅 기법: **랜덤 포레스트(Random Forest)**  


### 2.3 랜덤 포레스트 (Random Forest)
- 여러 개의 **결정 트리(Decision Tree)** 를 학습시킨 뒤,  
  예측을 모아 최종 결과를 결정하는 알고리즘  
- 각 트리는 데이터와 특성을 랜덤하게 선택 → 다양성을 확보  
- 장점:  
  - 단일 결정 트리보다 과적합 위험이 낮음  
  - 속성 중요도(Feature Importance)를 제공

### 2.4 Scikit-learn으로 랜덤 포레스트 실습

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# 데이터 불러오기
data = load_breast_cancer()
X, y = data.data, data.target

# 학습/테스트 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 랜덤 포레스트 모델 학습
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# 예측 및 평가
y_pred = rf.predict(X_test)
print("정확도:", accuracy_score(y_test, y_pred))

# 속성 중요도 출력
import pandas as pd
feature_importances = pd.Series(rf.feature_importances_, index=data.feature_names)
print(feature_importances.sort_values(ascending=False).head(10))

### 2.5 랜덤 포레스트의 장점과 한계
- **장점**  
  - 단일 결정 트리보다 일반화 성능이 높음  
  - 다양한 데이터셋에서 안정적인 성능  
  - 속성 중요도를 통해 해석 가능  

- **한계**  
  - 단일 트리에 비해 학습 속도가 느릴 수 있음  
  - 매우 많은 트리를 사용할 경우 계산량 증가  
  - 최신 알고리즘에 비해 예측 성능이 떨어짐

### ✅ 체크포인트
- 배깅은 데이터를 여러 번 샘플링하여 여러 모델을 학습시키는 방법이다.  
- 랜덤 포레스트는 여러 결정 트리를 조합한 대표적인 배깅 기법이다.  
- 랜덤 포레스트는 과적합 위험을 줄이고, 안정적이고 강력한 성능을 제공한다.  
- 속성 중요도를 통해 어떤 변수가 중요한지 확인할 수 있다.  


## 3. 부스팅 (Boosting)

### 3.1 부스팅 개념
- 배깅과 달리, **순차적으로 모델을 학습**시키는 방법  
- 각 단계에서 **이전 모델이 틀린 데이터를 더 잘 맞추도록** 다음 모델이 학습  
- 즉, **이전 모델의 오차(error)** 를 점점 줄여가며 전체 성능을 향상  
- 약한 학습기(Weak Learner) 여러 개를 **가중합(weighted sum)** 형태로 결합해  
  강한 학습기(Strong Learner)를 만든다  
- 대표 알고리즘: **Gradient Boosting, AdaBoost, XGBoost**

**데이터는 복원추출하지 않음**  
→ 모든 학습 단계에서 전체 데이터를 사용하지만, 각 샘플에 **가중치(weight)** 를 달리 부여하여  
**이전 모델이 잘못 예측한 샘플에 더 집중**


### 3.2 Gradient Boosting
- 기본 아이디어:  
  1. 초기 예측값(단순 평균)으로 시작  
  2. 모델이 틀린 부분(잔차, residual)을 계산 
  3. 이 잔차를 줄이는 방향으로 **새로운 트리(약한 모델)** 추가 
  4. 각 단계에서 **작은 학습률(learning rate)** 을 곱해 조금씩 보정  

👉 장점: 높은 정확도  
👉 단점: 순차 학습이라 학습 속도가 느림


$F_m(x) = F_{m-1}(x) + \eta \cdot h_m(x)$ 

- $F_m(x)$: m번째까지의 전체 모델  
- $h_m(x)$: 새로 추가되는 약한 모델  
- $\eta$: 학습률(learning rate) → 보정 강도를 조절

### 정리  

배깅(Bagging)은 여러 모델을 **독립적으로 병렬 학습**시켜  
결과를 평균(또는 다수결)하는 방식으로 **분산(Variance)** 을 줄이는 기법이고,  
 
부스팅(Boosting)은 모델을 **순차적으로 학습**시켜 **앞선 모델의 오차를 다음 모델이 보정**하도록  
만들어 **편향(Bias)** 을 줄이는 기법입니다.  

👉 배깅은 “다양성으로 안정화”,  
👉 부스팅은 “오차 보정으로 정밀화”를 노립니다.

In [ ]:
### Gradient Boosting 실습 (scikit-learn)

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_breast_cancer

# 데이터
data = load_breast_cancer()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 학습
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb.fit(X_train, y_train)

# 예측 및 평가
y_pred = gb.predict(X_test)
print("Gradient Boosting 정확도:", accuracy_score(y_test, y_pred))

### 3.3 XGBoost
- Gradient Boosting을 최적화한 라이브러리  
- 장점:  
  - 더 빠른 학습 속도  
  - 정규화(L1, L2) 지원 → 과적합 방지  
  - 대규모 데이터에서도 효율적  
- Kaggle 대회에서 자주 사용되는 강력한 알고리즘 

In [ ]:
### XGBoost 실습

import xgboost as xgb
from xgboost import XGBClassifier

# 모델 학습
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42, use_label_encoder=False, eval_metric="logloss")
xgb_model.fit(X_train, y_train)

# 예측 및 평가
y_pred = xgb_model.predict(X_test)
print("XGBoost 정확도:", accuracy_score(y_test, y_pred))

### 3.4 Gradient Boosting vs XGBoost
- **Gradient Boosting**: 구현이 단순, 기본적인 부스팅 개념 학습에 적합  
- **XGBoost**: 속도와 성능이 뛰어나 실무 및 대규모 데이터셋에 적합  


### ✅ 체크포인트
- 부스팅은 모델을 순차적으로 학습시켜 오차를 줄이는 방식이다.  
- Gradient Boosting은 기본적인 부스팅 알고리즘이다.  
- XGBoost는 Gradient Boosting을 개선하여 속도와 성능을 강화한 알고리즘이다.  
- 학습률과 트리 개수는 부스팅 알고리즘에서 중요한 하이퍼파라미터이다.  


## 4. 모델 평가와 비교



### 4.1 왜 모델 평가가 중요한가?
- 단일 성능 지표만 보면 과적합을 놓치기 쉽습니다.  
- 훈련 데이터뿐 아니라 **검증 데이터, 교차 검증 결과**로 모델을 평가해야 합니다.  
- 특히 **결정 트리 vs 랜덤 포레스트 vs 부스팅(XGBoost)** 모델의 차이를 비교할 수 있어야 합니다.  



### 4.2 교차 검증 (Cross Validation)
- 데이터를 여러 폴드로 나눠 학습과 평가를 반복 → 평균 성능 확인  
- 안정적이고 일반화된 성능 평가 가능

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb
import numpy as np

# 모델 정의
dt = DecisionTreeClassifier(max_depth=3, random_state=42)
rf = RandomForestClassifier(n_estimators=100, random_state=42)
gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42, use_label_encoder=False, eval_metric="logloss")

models = {"Decision Tree": dt, "Random Forest": rf, "Gradient Boosting": gb, "XGBoost": xgb_model}

# 교차 검증
for name, model in models.items():
    scores = cross_val_score(model, X, y, cv=5)
    print(f"{name} 평균 정확도: {np.mean(scores):.4f}")

### 4.3 과적합 확인
- **결정 트리**: 깊이가 깊어질수록 과적합 위험 ↑  
- **랜덤 포레스트**: 과적합 방지 효과 ↑  
- **부스팅(XGBoost)**: 높은 성능, 그러나 과적합 가능 → 학습률과 트리 수 조정 필요  

👉 해결책:  
- 교차 검증으로 안정적 평가  
- `max_depth`, `n_estimators`, `learning_rate` 등 하이퍼파라미터 조정  



### 4.4 성능 비교 예시 출력
- 보통 결과는 다음과 같이 나올 수 있음 (데이터셋에 따라 다름)  

| 모델               | 평균 정확도 |
|--|-|
| Decision Tree      | 0.90        |
| Random Forest      | 0.95        |
| Gradient Boosting  | 0.96        |
| XGBoost            | 0.97        |



### ✅ 체크포인트
- 모델은 반드시 교차 검증으로 평가해야 한다.  
- 결정 트리는 과적합되기 쉬운 반면, 랜덤 포레스트와 부스팅은 일반화 성능이 더 높다.  
- XGBoost는 대규모 데이터셋에서 뛰어난 성능을 보인다.  
- 하이퍼파라미터 튜닝을 통해 성능을 추가로 개선할 수 있다.  


### 결정 트리와 앙상블 기법 – 실습 문제

### 문제 1. 결정 트리 학습 및 시각화
Breast Cancer 데이터셋을 불러와서,  
`max_depth=3`인 결정 트리를 학습하고 시각화하세요.  

<details>
<summary>정답 보기</summary>

```python
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt

data = load_breast_cancer()
X, y = data.data, data.target

model = DecisionTreeClassifier(max_depth=3, random_state=42)
model.fit(X, y)

plt.figure(figsize=(12,6))
plot_tree(model, feature_names=data.feature_names, class_names=data.target_names, filled=True)
plt.show()
```
</details>



In [ ]:
# 여기에 정답을 작성하세요
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt

data = load_breast_cancer()
X, y = data.data, data.target

### 문제 2. 랜덤 포레스트 학습 및 정확도 평가
Breast Cancer 데이터셋을 사용하여  
`n_estimators=100`인 랜덤 포레스트 분류기를 학습하고 테스트 정확도를 구하세요.  

<details>
<summary>정답 보기</summary>

```python
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
print("정확도:", accuracy_score(y_test, y_pred))
```
</details>

In [ ]:
# 여기에 정답을 작성하세요
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt

data = load_breast_cancer()
X, y = data.data, data.target

### 문제 3. 속성 중요도 확인
문제 2에서 학습한 랜덤 포레스트 모델의 **상위 5개 속성 중요도**를 출력하세요.  

속성 중요도란?
- **속성(feature) 중요도**는 모델이 **예측을 할 때 각 입력 변수가 얼마나 중요한 역할을 했는지**를 나타내는 값입니다.  
- 값이 높을수록 그 특성이 **결과에 더 큰 영향을 미쳤다**는 의미입니다.  
- 트리 기반 모델(의사결정나무, 랜덤포레스트, XGBoost 등)은 학습 과정에서  
  데이터를 분할할 때 **불순도(impurity)** 를 얼마나 줄였는지를 기준으로  
  자동으로 각 특성의 중요도를 계산합니다.  

확인 방법
- 학습된 모델 객체의 속성 `feature_importances_` 를 확인하면 됩니다.  
- 이 값은 각 특성의 상대적 중요도를 나타내며,  
  합계가 1(=100%)이 되도록 정규화되어 있습니다.  

<details>
<summary>정답 보기</summary>


```python
import pandas as pd

feature_importances = pd.Series(rf.feature_importances_, index=data.feature_names)
print(feature_importances.sort_values(ascending=False).head(5))
```
</details>



In [ ]:
# 여기에 정답을 작성하세요
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt

data = load_breast_cancer()
X, y = data.data, data.target

### 문제 4. Gradient Boosting 적용
Breast Cancer 데이터셋에서 `learning_rate=0.1`, `n_estimators=100`인  
GradientBoostingClassifier를 학습하고 정확도를 출력하세요.  

<details>
<summary>정답 보기</summary>

```python
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
gb.fit(X_train, y_train)

y_pred = gb.predict(X_test)
print("Gradient Boosting 정확도:", accuracy_score(y_test, y_pred))
```
</details>



In [ ]:
# 여기에 정답을 작성하세요
from sklearn.datasets import load_breast_cancer
from sklearn.tree import plot_tree
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt

data = load_breast_cancer()
X, y = data.data, data.target

### 문제 5. XGBoost 적용 및 비교
Breast Cancer 데이터셋에서 XGBoost 모델을 학습하고,  
Gradient Boosting과 정확도를 비교하세요.  

<details>
<summary>정답 보기</summary>

```python
from xgboost import XGBClassifier

xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3,
                          random_state=42, use_label_encoder=False, eval_metric="logloss")
xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)
print("XGBoost 정확도:", accuracy_score(y_test, y_pred))
```
</details>

In [ ]:
# 여기에 정답을 작성하세요
from sklearn.datasets import load_breast_cancer
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

data = load_breast_cancer()
X, y = data.data, data.target

### 문제 6. 교차 검증으로 모델 비교
결정 트리, 랜덤 포레스트, Gradient Boosting, XGBoost 모델을  
5겹 교차 검증하여 평균 정확도를 각각 출력하세요.  

<details>
<summary>정답 보기</summary>

```python
from sklearn.model_selection import cross_val_score
import numpy as np

models = {
    "Decision Tree": DecisionTreeClassifier(max_depth=3, random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42),
    "XGBoost": XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42, 
                              use_label_encoder=False, eval_metric="logloss")
}

for name, model in models.items():
    scores = cross_val_score(model, X, y, cv=5)
    print(f"{name} 평균 정확도: {np.mean(scores):.4f}")
```
</details>



In [ ]:
# 여기에 정답을 작성하세요
from sklearn.datasets import load_breast_cancer
from xgboost import XGBClassifier
import matplotlib.pyplot as plt

data = load_breast_cancer()
X, y = data.data, data.target

### ✅ 체크포인트
- 결정 트리는 이해하기 쉽지만 과적합 위험이 크다.  
- 랜덤 포레스트는 배깅 기법으로 안정적 성능을 낸다.  
- Gradient Boosting은 잔차를 줄이며 학습하는 강력한 기법이다.  
- XGBoost는 최적화된 부스팅 알고리즘으로, 속도와 성능이 뛰어나다.  
- 교차 검증을 통해 모델의 일반화 성능을 안정적으로 비교할 수 있다.  


### Kaggle 경진대회 도전

**대회 주소 :** https://www.kaggle.com/competitions/titanic/overview

**목표 :** 전처리 방법 변경 및 모델을 최적화하여 제출 후 스코어 0.82 이상 도달

**주의사항 :** 파일 제출 횟수가 하루 10번으로 제한

**참고 :**   아래 Baseline 코드는 참고만 하시고 되도록 직접 모든 과정을 해보시길 권장드립니다.

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# 데이터 불러오기
train = pd.read_csv('data/titanic.csv')
test = pd.read_csv('data/test.csv')

# 데이터 전처리
def preprocess_data(df):
    # 결측치 처리
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])
    df['Fare'] = df['Fare'].fillna(df['Fare'].mean())
    
    # 범주형 변수 처리
    df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
    df['Embarked'] = df['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})
    
    # 필요한 특성 선택
    features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
    return df[features]

# 학습 데이터 전처리
X = preprocess_data(train)
y = train['Survived']

# 학습 데이터와 검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# RandomForest 모델 생성 및 학습
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# 검증 데이터로 예측
val_pred = rf_model.predict(X_val)

# 모델 성능 평가
print('검증 데이터 정확도:', accuracy_score(y_val, val_pred))
print('\n분류 보고서:')
print(classification_report(y_val, val_pred))

# 테스트 데이터 예측
test_processed = preprocess_data(test)
test_pred = rf_model.predict(test_processed)

# 제출 파일 생성
submission = pd.DataFrame({
    'PassengerId': test['PassengerId'],
    'Survived': test_pred
})
submission.to_csv('submission.csv', index=False)
print('\n제출 파일이 생성되었습니다.')


검증 데이터 정확도: 0.8212290502793296

분류 보고서:
              precision    recall  f1-score   support

           0       0.83      0.87      0.85       105
           1       0.80      0.76      0.78        74

    accuracy                           0.82       179
   macro avg       0.82      0.81      0.81       179
weighted avg       0.82      0.82      0.82       179


제출 파일이 생성되었습니다.
